In [2]:
# from IPython.display import clear_output
# # Install required packages.
# #%%capture

# !pip install torch-geometric
# !pip install sentence_transformers

# clear_output()

In [1]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import GCNConv, SAGEConv, to_hetero
import torch_geometric.transforms as T
from torch.utils.data import DataLoader

In [2]:
dataset_path = '/tmp/'
dataset = MovieLens(root=dataset_path)

In [3]:
import numpy as np

In [4]:
dataset[0]

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
  }
)

In [5]:
device = 'mps'

data = dataset[0].to(device)

In [6]:
# Add user node features for message passing:
data['user'].x = torch.eye(data['user'].num_nodes, device=device)
del data['user'].num_nodes

# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

In [7]:
data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
  },
  (movie, rev_rates, user)={ edge_index=[2, 100836] }
)

In [8]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)(data)

In [9]:
train_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ x=[610, 610] },
  (user, rates, movie)={
    edge_index=[2, 80670],
    edge_label=[80670],
    edge_label_index=[2, 80670],
  },
  (movie, rev_rates, user)={ edge_index=[2, 80670] }
)

In [10]:
torch.all(train_data['user', 'rates', 'movie'].edge_label_index == train_data['user', 'rates', 'movie'].edge_index)

tensor(True, device='mps:0')

In [11]:
train_data.edge_index_dict

{('user',
  'rates',
  'movie'): tensor([[ 443,  355,  155,  ...,  305,  403,  447],
         [ 325, 1108,   18,  ..., 7207,  172, 8846]], device='mps:0'),
 ('movie',
  'rev_rates',
  'user'): tensor([[ 325, 1108,   18,  ..., 7207,  172, 8846],
         [ 443,  355,  155,  ...,  305,  403,  447]], device='mps:0')}

In [12]:
weight = torch.bincount(train_data['user', 'rates','movie'].edge_label)
print(weight)

weight = weight.max() / weight
weight

tensor([ 1080,  3703, 10464, 26620, 28234, 10569], device='mps:0')


tensor([26.1426,  7.6246,  2.6982,  1.0606,  1.0000,  2.6714], device='mps:0')

In [29]:
class WeightedMSELoss(torch.nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

    def forward(self, pred, target):
        weight = self.weights[target].to(pred.dtype)
        return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()


class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [32]:
loss_fn = WeightedMSELoss(weight)

def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'rates','movie'].edge_label_index)
    target = train_data['user', 'rates','movie'].edge_label
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'rates','movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'rates','movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

In [15]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from tqdm import tqdm


wandb.login()

wandb: Currently logged in as: betev-vanya. Use `wandb login --relogin` to force relogin


True

In [16]:
run = wandb.init(project="GraphSAGE_MonieLens")

In [17]:
class MovieLensModule(pl.LightningModule):
    def __init__(self, learning_rate=0.01):
        super().__init__()
        self.model = Model(hidden_channels=64).to(device)
        self.learning_rate = learning_rate
        self.loss_fn = WeightedMSELoss(weight)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        result = self.model(x_dict, edge_index_dict, edge_label_index)
        return result

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optim

    def training_step(self, train_batch, batch_idx):
        pred = self.model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'rates','movie'].edge_label_index)
        target = train_data['user', 'rates','movie'].edge_label
        loss = self.loss_fn(pred, target)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        data = test_data
        pred = self.model(data.x_dict, data.edge_index_dict,
                 data['user', 'rates','movie'].edge_label_index)
        pred = pred.clamp(min=0, max=5)
        target = data['user', 'rates','movie'].edge_label.float()
        rmse = F.mse_loss(pred, target).sqrt()
        self.log("val_rmse", float(rmse), prog_bar=True)

In [25]:
wandb_logger = WandbLogger(log_model='all') 
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_rmse", mode="min")

trainer = pl.Trainer(logger=wandb_logger, callbacks=[checkpoint_callback],
                     accelerator='mps', max_epochs=400)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [22]:
# module = MovieLensModule()

# lr_finder = trainer.tuner.lr_find(module, DataLoader([0]), DataLoader([0]), min_lr=0.001, max_lr=0.1)

In [28]:
# lr_finder.suggestion()

In [27]:
# module = MovieLensModule(learning_rate=0.095)#lr_finder.suggestion())

# trainer.fit(module, DataLoader([0]), DataLoader([0]))

In [33]:
model = Model(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(1, 401):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    if epoch % 25 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 025, Loss: 5.9824, Train: 1.7718, Val: 1.7834, Test: 1.7823
Epoch: 050, Loss: 3.9647, Train: 1.2879, Val: 1.2981, Test: 1.3001
Epoch: 075, Loss: 3.5655, Train: 1.1310, Val: 1.1421, Test: 1.1390
Epoch: 100, Loss: 3.2412, Train: 1.1334, Val: 1.1550, Test: 1.1494
Epoch: 125, Loss: 3.0059, Train: 1.1003, Val: 1.1399, Test: 1.1306
Epoch: 150, Loss: 2.8493, Train: 1.0830, Val: 1.1226, Test: 1.1152
Epoch: 175, Loss: 2.6568, Train: 1.0306, Val: 1.0824, Test: 1.0738
Epoch: 200, Loss: 2.5051, Train: 1.0549, Val: 1.1173, Test: 1.1109
Epoch: 225, Loss: 2.3417, Train: 1.0407, Val: 1.1179, Test: 1.1116
Epoch: 250, Loss: 2.2335, Train: 0.9649, Val: 1.0522, Test: 1.0438
Epoch: 275, Loss: 2.1425, Train: 0.9811, Val: 1.0808, Test: 1.0742
Epoch: 300, Loss: 2.0787, Train: 1.0018, Val: 1.1071, Test: 1.1017
Epoch: 325, Loss: 2.1337, Train: 1.0417, Val: 1.1482, Test: 1.1448
Epoch: 350, Loss: 2.0550, Train: 0.9781, Val: 1.0943, Test: 1.0891
Epoch: 375, Loss: 1.9669, Train: 0.9308, Val: 1.0603, Test: 1.

In [34]:
torch.save(model.state_dict(), 'weights/epoch_400.pth')